# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [53]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [54]:
%run sql_queries.py

In [55]:
%run create_tables.py

OperationalError: database "sparkifydb" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [56]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [57]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [58]:
song_files = get_files('data/song_data')

In [59]:
filepath = song_files[0]

In [60]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [61]:
#song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()[0]
song_data

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [62]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [63]:
#after error, resolved by utilizing code from knowledge: https://knowledge.udacity.com/questions/936636
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [64]:
log_files = get_files('data/log_data')

In [65]:
filepath = log_files[0]

In [66]:
df = pd.read_json(filepath, lines=True)
#df['ts'] = df['ts'].astype(str)
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 18 columns):
artist           82 non-null object
auth             98 non-null object
firstName        91 non-null object
gender           91 non-null object
itemInSession    98 non-null int64
lastName         91 non-null object
length           82 non-null float64
level            98 non-null object
location         91 non-null object
method           98 non-null object
page             98 non-null object
registration     91 non-null float64
sessionId        98 non-null int64
song             82 non-null object
status           98 non-null int64
ts               98 non-null datetime64[ns]
userAgent        91 non-null object
userId           98 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(12)
memory usage: 13.9+ KB


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [67]:
#referenced: https://www.geeksforgeeks.org/pandas-query-method/
#df = df.query('page == "NextSong"')
df=df[df['page'] == 'NextSong']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 95
Data columns (total 18 columns):
artist           82 non-null object
auth             82 non-null object
firstName        82 non-null object
gender           82 non-null object
itemInSession    82 non-null int64
lastName         82 non-null object
length           82 non-null float64
level            82 non-null object
location         82 non-null object
method           82 non-null object
page             82 non-null object
registration     82 non-null float64
sessionId        82 non-null int64
song             82 non-null object
status           82 non-null int64
ts               82 non-null datetime64[ns]
userAgent        82 non-null object
userId           82 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(12)
memory usage: 12.2+ KB


In [68]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,2018-11-22 00:03:52.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,2018-11-22 00:09:10.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [69]:
#Referenced https://www.geeksforgeeks.org/python-pandas-to_datetime/
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
t = df.copy()
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,2018-11-22 00:03:52.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,2018-11-22 00:09:10.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,2018-11-22 00:12:06.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,2018-11-22 00:15:41.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,2018-11-22 00:23:40.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [70]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 95
Data columns (total 18 columns):
artist           82 non-null object
auth             82 non-null object
firstName        82 non-null object
gender           82 non-null object
itemInSession    82 non-null int64
lastName         82 non-null object
length           82 non-null float64
level            82 non-null object
location         82 non-null object
method           82 non-null object
page             82 non-null object
registration     82 non-null float64
sessionId        82 non-null int64
song             82 non-null object
status           82 non-null int64
ts               82 non-null datetime64[ns]
userAgent        82 non-null object
userId           82 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(12)
memory usage: 12.2+ KB


In [71]:
#Referenced: https://sparkbyexamples.com/pandas/append-pandas-dataframes-using-for-loop/
#time_data = []
#for i in t:
#    time_data.append([i, i.hour, i.day, i.week, i.month, i.year, i.day_name()])
#column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
#kept getting an error on the datatype so resorted to running one at a time 

time_data = pd.DataFrame()
time_data['start_time'] = t['ts']
time_data['hour'] = t['ts'].dt.hour
time_data['day'] = t['ts'].dt.day
time_data['week'] = t['ts'].dt.week
time_data['year'] = t['ts'].dt.year
time_data['month'] = t['ts'].dt.month
time_data['weekday'] = t['ts'].dt.weekday

time_df = time_data.copy()
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 95
Data columns (total 7 columns):
start_time    82 non-null datetime64[ns]
hour          82 non-null int64
day           82 non-null int64
week          82 non-null int64
year          82 non-null int64
month         82 non-null int64
weekday       82 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 5.1 KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [72]:
#referenced: https://stackoverflow.com/questions/2209169/using-postgres-in-a-web-app-transaction-aborted-errors

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
time_table_insert = ("INSERT into time (start_time, hour, day, week, month, year,weekday) VALUES(%s, %s, %s, %s, %s, %s,%s) ON CONFLICT DO NOTHING ")


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [73]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 95
Data columns (total 5 columns):
userId       82 non-null object
firstName    82 non-null object
lastName     82 non-null object
gender       82 non-null object
level        82 non-null object
dtypes: object(5)
memory usage: 3.8+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [74]:

for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [76]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [1]:
%run etl.py

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

IndexError: tuple index out of range